In [1]:
import numpy as np
import scipy
from numba import njit, prange
from numba.core import types
from numba.typed import Dict
from numba.types import boolean

In [2]:
def gen_graph_pure(nverts=3e3, dims=3):
    vertices = {i: np.random.uniform(size=dims) for i in range(int(nverts))}
    return vertices

In [3]:
vertex_array = types.f8[:]
@njit()
def gen_graph_compiled(nverts=3e3, dims=3):
    vertices = Dict.empty(
        key_type = types.i8,
        value_type = vertex_array
    )
    for i in prange(int(nverts)):
        vertices[i] = np.random.uniform(0, 1, dims)
    return vertices
        

In [4]:
%%time
verti = gen_graph_pure()

CPU times: user 17.2 ms, sys: 8.98 ms, total: 26.2 ms
Wall time: 16 ms


In [5]:
%%time
vertc = gen_graph_compiled()

CPU times: user 437 ms, sys: 7.58 ms, total: 445 ms
Wall time: 446 ms


In [6]:
%%time
vertc = gen_graph_compiled()

CPU times: user 659 µs, sys: 55 µs, total: 714 µs
Wall time: 716 µs


In [7]:
def choose_edges_pure(vertices: dict, min_connections=2, max_connections=-1):
    edges = {}
    maxxed = []
    for i in vertices.keys():
        dropped = np.array([i]+maxxed)
        other_verts = np.delete(np.array([*vertices]), dropped)
        nedges = min_connections
        if max_connections < 0:
            nedges = min_connections
        else:
            nedges = np.random.randint(min_connections, max_connections)
        these_edges = np.random.choice(other_verts, nedges, replace=False)
        edges[i] = these_edges
        if len(edges[i]) == max_connections:
            maxxed.append(i)
        for j in these_edges:
            if (j in list(edges.keys())):
                edges[j] = np.concatenate((edges[j],np.array([i])))
                if len(edges[j]) == max_connections:
                    maxxed.append(j)
            else:
                edges[j] = np.array([i])

    return edges

In [8]:
%%time
edges = choose_edges_pure(verti)

CPU times: user 1.26 s, sys: 0 ns, total: 1.26 s
Wall time: 1.27 s


In [9]:
vertex_array = types.i8[:]
@njit
def choose_edges_compiled(vertices: Dict, min_connections=2, max_connections=-1):
    edges = Dict.empty(
        key_type = types.i8,
        value_type = vertex_array
    )
    maxxed = []
    verts = np.array(list(vertices.keys()))
    for i in vertices.keys():
        dropped = np.array([i]+maxxed)
        other_verts = np.delete(verts, dropped)
        if max_connections < 0:
            nedges = min_connections
        else:
            nedges = np.random.randint(min_connections, max_connections)
        these_edges = np.random.choice(other_verts, nedges, replace=False)
        edges[i] = these_edges
        if len(edges[i]) == max_connections:
            maxxed.append(i)
        for j in these_edges:
            if (j in list(edges.keys())):
                edges[j] = np.concatenate((edges[j],np.array([i])))
                if len(edges[j]) == max_connections:
                    maxxed.append(j)
            else:
                edges[j] = np.array([i])

    return edges
    

In [10]:
%%time
edgesc = choose_edges_compiled(vertc)

CPU times: user 1.42 s, sys: 6.39 ms, total: 1.43 s
Wall time: 1.43 s


In [25]:
source, target = np.random.choice(list(vertc.keys()), 2, False)
print(f'Source vertex: {source}, {vertc[source]}')
print(f'Target vertex: {target}, {vertc[target]}')

Source vertex: 2835, [0.92713067 0.288936   0.19704476]
Target vertex: 2599, [0.19033441 0.29087523 0.44477602]


In [12]:
def get_mindist(dist, Q):
    dists = np.empty(len(dist), np.float64)
    for i in prange(dists.shape[0]):
        a = boolean(i in Q)
        dists[i] = dist[i] * a + np.float64(10) * (1-a)
    return np.argmin(dists)
            

def dijkstra_pure(verts, edges, source, target):
    dist = {}
    prev = {}
    Q = list(verts.keys())
    for v in verts:
        dist[v] = 10
        prev[v] = -1
    dist[source] = 0
    while len(Q) > 1:
        u = get_mindist(dist, Q)
        Q.remove(u)
        if u == target:
            return dist, prev
        for v in edges[u]:
            alt = dist[u] + np.linalg.norm(verts[u] - verts[v])
            a = boolean(alt < dist[v])
            dist[v] = alt * a + dist[v] * (1-a)
            prev[v] = u * a + prev[v] * (1-a)
    return dist, prev
        

In [13]:
@njit(parallel=True)
def get_mindistc(dist, Q):
    dists = np.empty(len(dist), np.float64)
    for i in prange(dists.shape[0]):
        a = boolean(i in Q)
        dists[i] = dist[i] * a + np.float64(10) * (1-a)
    return np.int64(np.argmin(dists))

@njit(parallel=True)
def dijkstra_compiled(verts, edges, source, target):
    dist = Dict.empty(
        key_type = types.i8,
        value_type = types.f8
    )
    prev = Dict.empty(
        key_type = types.i8,
        value_type = types.i8
    )
    Q = list(verts.keys())
    for v in range(len(verts)):
        dist[v] = np.float(10)
        prev[v] = np.int64(-1)
    dist[source] = 0
    while len(Q) > 1:
        u = get_mindistc(dist, Q)
        Q.remove(u)
        if u == target:
            return dist, prev
        for i in prange(len(edges[u])):
            v = edges[u][i]
            alt = dist[u] + np.linalg.norm(verts[u] - verts[v])
            a = boolean(alt < dist[v])
            dist[v] = alt * a + dist[v] * (1-a)
            prev[v] = u * a + prev[v] * (1-a)
    return dist, prev


In [14]:
def path_dijkstra(prev, source, target):
    s = []
    u = target
    while u != source:
        s.append(u)
        u = prev[u]
    s.append(source)
    return s

In [26]:
%%time
dist, prev = dijkstra_pure(vertc, edgesc, source, target)

CPU times: user 55.9 s, sys: 1.22 ms, total: 55.9 s
Wall time: 56 s


In [33]:
%%time
distc, prevc = dijkstra_compiled(vertc, edgesc, source, target)

CPU times: user 6.62 s, sys: 519 ms, total: 7.14 s
Wall time: 689 ms


In [31]:
path_dijkstra(prev, source, target)

[2599, 2052, 16, 1589, 89, 9, 2210, 582, 2010, 123, 2835]

In [32]:
path_dijkstra(prevc, source, target)

[2599, 2052, 16, 1589, 89, 9, 2210, 582, 2010, 123, 2835]

In [1]:
3e3

3000.0